# Install and import the python library we need
Click the play button (or Shift + Enter)

In [3]:
!pip install polars --upgrade
import polars as pl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.9.0
    Uninstalling polars-1.9.0:
      Successfully uninstalled polars-1.9.0


# Load/prep the Spotify playlist data I've aggregated
1. Download and save the raw [wcs_dj_spotify_playlists.parquet](https://github.com/ThomasMAhern/WCS_playlist_analysis/blob/main/wcs_dj_spotify_playlists.parquet) file from github
2. Upload it into the folder icon to the <left<
3. Run the cell below

In [65]:
pl.Config.set_tbl_rows(100).set_fmt_str_lengths(100)

regex_year_first = r'\d{2,4}[.\-/ ]?\d{1,2}[.\-/ ]?\d{1,2}'
regex_year_last = r'\d{1,2}[.\-/ ]?\d{1,2}[.\-/ ]?\d{2,4}'
regex_year_abbreviated = r"'\d{2}"

def wcs_specific(df_):
  '''given a df, filter to the records most likely to be west coast swing related'''
  return (df_
          .filter(~(pl.col('name').str.contains(regex_year_first)
                  |pl.col('name').str.contains(regex_year_last)
                  |pl.col('name').str.contains(regex_year_abbreviated)
                  |pl.col('name').str.to_lowercase().str.contains('wcs|social|party|oirée|west coast|routine|blues|practice|practise|bpm|swing|novice|intermediate|comp|musicality|timing|pro show')))
      )

df = (pl.read_parquet('wcs_dj_spotify_playlists.parquet')

      #makes a new column filled with a date - this is good indicator if there was a set played
      .with_columns(extracted_date = pl.concat_list(pl.col('name').str.extract_all(regex_year_last),
                                                    pl.col('name').str.extract_all(regex_year_last),
                                                    pl.col('name').str.extract_all(regex_year_abbreviated),)
                                       .list.unique().list.sort(),
                    song = pl.concat_str('track.name', pl.lit(' - https://open.spotify.com/track/'), 'track.id', ignore_nulls=True),
                    region = pl.col('location').str.split(' - ').list.get(0),)

      #gets the counts of djs, playlists, and geographic regions a song is found in
      .with_columns(num_djs = pl.n_unique('owner.display_name').over(pl.col('song')),
                    num_playlists = pl.n_unique('name').over(pl.col('song')),
                    num_regions = pl.n_unique('region').over('song'),
                    regions = pl.col('region').over('song', mapping_strategy='join')
                                  .list.unique()
                                  .list.drop_nulls()
                                  .list.sort()
                                  .list.join(', '),)
      )

print(f"{df.select(pl.concat_str('track.name', pl.lit(' - '), 'track.id')).unique().shape[0]:,} Songs ({df.pipe(wcs_specific).select(pl.concat_str('track.name', pl.lit(' - '), 'track.id')).unique().shape[0]:,} wcs specific)")
print(f"{df.select('track.artists.id').unique().shape[0]:,} Artists ({df.pipe(wcs_specific).select('track.artists.id').unique().shape[0]:,} wcs specific)")
print(f"{df.select('name').unique().shape[0]:,} Playlists ({df.pipe(wcs_specific).select('name').unique().shape[0]:,} wcs specific)\n\n")
# display(df.sample(100))

126,264 Songs (93,982 wcs specific)
44,100 Artists (36,253 wcs specific)
8,154 Playlists (3,971 wcs specific)




# What's popular music others play, but I don't?
* input your display name ("Kasia Stepek")
* or spotify user id ("1185428002" from `open.spotify.com/user/1185428002`)

This will output a list of the top 500 songs most others have, but that don't appear in your spotify library.

In [66]:
dj_id = 'Kasia Stepek'


pl.Config.set_tbl_rows(10) #gives us a small output while still showing total rows
not_my_music = (df
                #  .pipe(wcs_specific)
                .filter(~pl.col('spotify').str.contains(dj_id)
                        | ~pl.col('owner.display_name').str.contains(dj_id))
                .filter(pl.col('num_djs') > 5,
                        pl.col('num_playlists') > 5)
                .select('song', 'num_djs', 'num_playlists', 'num_regions', 'regions')
                .unique()
                .sort('num_playlists', descending=True)
                )
display(not_my_music)

for song, *_ in not_my_music.head(500).iter_rows():
  print(song)

song,num_djs,num_playlists,num_regions,regions
str,u32,u32,u32,str
"""Chunks - https://open.spotify.com/track/3lFCbvyrJ7soEgTjIkELMT""",108,296,5,"""Asia, Europe, MENA, USA"""
"""California King - https://open.spotify.com/track/180OrhCzFdX7Pyhri6AerI""",103,271,6,"""Asia, Europe, MENA, Oceania, USA"""
"""Gravity - https://open.spotify.com/track/5EjG2jgzovOhN3OzWCcbdu""",93,236,5,"""Asia, Europe, MENA, USA"""
"""Intrigued - https://open.spotify.com/track/1lbgPy6qUbszFfZwQVec97""",100,236,6,"""Asia, Europe, MENA, Oceania, USA"""
"""Bloodstream - https://open.spotify.com/track/5v4sZRuvWDcisoOk1PFv6T""",122,228,5,"""Asia, Europe, MENA, USA"""
…,…,…,…,…
"""Dirty Paws - https://open.spotify.com/track/5g7rJvWYVrloJZwKiShqlS""",6,6,2,"""Europe"""
"""A Thousand Miles - https://open.spotify.com/track/6Qm9MaditCcx5V62rVHYqu""",6,6,4,"""Europe, MENA, USA"""
"""Can't Say No - https://open.spotify.com/track/0AHpuVdVEbQu8bzGUieSKi""",6,6,2,"""MENA"""


Chunks - https://open.spotify.com/track/3lFCbvyrJ7soEgTjIkELMT
California King - https://open.spotify.com/track/180OrhCzFdX7Pyhri6AerI
Gravity - https://open.spotify.com/track/5EjG2jgzovOhN3OzWCcbdu
Intrigued - https://open.spotify.com/track/1lbgPy6qUbszFfZwQVec97
Bloodstream - https://open.spotify.com/track/5v4sZRuvWDcisoOk1PFv6T
The Mates of Soul (Remastered) - https://open.spotify.com/track/6hBTkWmjb8YsOzINHebJ3e
Vibe - https://open.spotify.com/track/1pDDhg9lxPMxwC3q6Kngg2
Quand on arrive en ville - https://open.spotify.com/track/3i968ZBc1IdtDZEL9zuKWP
Crush On You - https://open.spotify.com/track/4YkhN5nT1FkmmqbxzD8Ljh
What A Wonderful World - https://open.spotify.com/track/4AsCoWKZ7i1c5WY5xWkhz9
Surrender - https://open.spotify.com/track/32amQ0lTNaApjW8IgeIBpI
Aliens Need Love Too - https://open.spotify.com/track/23OQAJVJ0VqXYqveHZRUyv
Josephine - https://open.spotify.com/track/2V5vmvIwxqVizuaYJgrmlE
6's to 9's - https://open.spotify.com/track/4DAaQ5InUO23d8yNRbB0Yj
SexyBack - htt

# What's the music that only I play?

This will output the top 500 songs that are unique to you... it may be blank

In [67]:
pl.Config.set_tbl_rows(10)
only_i_play = (df
              #  .pipe(wcs_specific)
              .filter(pl.col('num_djs').eq(1)
                      &(pl.col('spotify').str.contains(dj_id)
                        |pl.col('owner.display_name').str.contains(dj_id))
                     )
              .select('song', 'num_djs', 'num_playlists', 'num_regions', 'regions')
              .unique()
              .sort('num_playlists', descending=True)
              )

display(only_i_play)

for song, *_ in only_i_play.head(500).iter_rows():
  print(song)


song,num_djs,num_playlists,num_regions,regions
str,u32,u32,u32,str
"""Oh Brother - https://open.spotify.com/track/2J8pnml4JaVufffb0mYbd5""",1,7,1,"""Europe"""
"""Retrograde - https://open.spotify.com/track/6llZPoHpm2nlceuNAwzvfV""",1,7,1,"""Europe"""
"""Blowback - https://open.spotify.com/track/07Mrtc712SH15xNPsjYTiD""",1,7,1,"""Europe"""
"""Loverboy - Radio Edit - https://open.spotify.com/track/0eE77LrpvsdNjszeEF7qCG""",1,6,1,"""Europe"""
"""Sleep - https://open.spotify.com/track/3MBNKrBYObu0nQrm3Na4ko""",1,6,1,"""Europe"""
…,…,…,…,…
"""I Appear Missing - https://open.spotify.com/track/6mfKEPTYiBAYZ9z0429jsp""",1,1,1,"""Europe"""
"""Save a Little Love for Me - https://open.spotify.com/track/6lqxlJWNaykLli5cIXMf7l""",1,1,1,"""Europe"""
"""Súbete (feat. Don Omar) - Remix Version - https://open.spotify.com/track/74elqOJCRusTANHlY5wdLT""",1,1,1,"""Europe"""


Oh Brother - https://open.spotify.com/track/2J8pnml4JaVufffb0mYbd5
Retrograde - https://open.spotify.com/track/6llZPoHpm2nlceuNAwzvfV
Blowback - https://open.spotify.com/track/07Mrtc712SH15xNPsjYTiD
Loverboy - Radio Edit - https://open.spotify.com/track/0eE77LrpvsdNjszeEF7qCG
Sleep - https://open.spotify.com/track/3MBNKrBYObu0nQrm3Na4ko
Off the Grid (feat. Khalid) - Autograf Remix - https://open.spotify.com/track/2U4oV7MzeaB9NCAlh6pVBQ
Vibes - Tigertown Remix - https://open.spotify.com/track/1CgUldlwNfY2JQY3sTEwPn
Friends - https://open.spotify.com/track/5Hw98Jnc3teIR42EoEMM93
Blame - https://open.spotify.com/track/0Cl5kShLrsbvLxuwfIBEnT
Out Of Reach - From "Bridget Jones's Diary" - https://open.spotify.com/track/7lOjYEj3Ga1TNwv8f49U0g
+ La vie - https://open.spotify.com/track/6UWcp6ASpVIvUWX5eZMtp2
dddd - https://open.spotify.com/track/6EFUBilMIbA3TOw0i3AgPH
Spirits - https://open.spotify.com/track/7GJ572zvnU94VZVrj5XRtb
Yesterday - https://open.spotify.com/track/2wIkeX1x52W9aanDv7IWD

# Most popular songs only played by DJs in Europe

In [71]:
pl.Config.set_tbl_rows(10)
europe = (df
          #  .pipe(wcs_specific)
          .filter(pl.col('regions') == 'Europe')
          .select('song', 'num_djs', 'num_playlists', 'num_regions', 'regions')
          .unique()
          .sort('num_djs', descending=True)
          )

display(europe)

for song, *_ in europe.head(500).iter_rows():
  print(song)

song,num_djs,num_playlists,num_regions,regions
str,u32,u32,u32,str
"""3 to 1 - https://open.spotify.com/track/1n1mBqf22FPTfM8OTD7ToW""",38,76,2,"""Europe"""
"""Colors - https://open.spotify.com/track/6J3HNhDBIwEX11bX8hzx5u""",32,64,2,"""Europe"""
"""Let Me Love You - https://open.spotify.com/track/4pdPtRcBmOSQDlJ3Fk945m""",28,37,2,"""Europe"""
"""Reste - https://open.spotify.com/track/3ebaeB2BdCEgZmwKv7p2r5""",28,58,2,"""Europe"""
"""The Heart Wants What It Wants - https://open.spotify.com/track/7pJgjBf82BrUQ3z7HdQvW1""",27,35,2,"""Europe"""
…,…,…,…,…
"""Red Hot Band - https://open.spotify.com/track/4fsPzmoXXtb9kKWSCOqKQ6""",1,1,1,"""Europe"""
"""Frühlingsstimmenwalzer - https://open.spotify.com/track/4WVazvT2DEMXjW1m06uooS""",1,1,1,"""Europe"""
"""IMMABEHOME - https://open.spotify.com/track/6R0b4L13kDkQczs8E1C4sS""",1,1,1,"""Europe"""


3 to 1 - https://open.spotify.com/track/1n1mBqf22FPTfM8OTD7ToW
Colors - https://open.spotify.com/track/6J3HNhDBIwEX11bX8hzx5u
Let Me Love You - https://open.spotify.com/track/4pdPtRcBmOSQDlJ3Fk945m
Reste - https://open.spotify.com/track/3ebaeB2BdCEgZmwKv7p2r5
The Heart Wants What It Wants - https://open.spotify.com/track/7pJgjBf82BrUQ3z7HdQvW1
Cross Me (feat. Chance the Rapper & PnB Rock) - https://open.spotify.com/track/543bCW2ruMPmxUBWirQ3MR
Loverboy - https://open.spotify.com/track/2ks1i7dumYRliNhPuxzQLm
Let It Go - https://open.spotify.com/track/40EB7ABUO6MoWMUwPKptJ7
Bad Liar - https://open.spotify.com/track/2RSHsoi04658QL5xgQVov3
Dusk Till Dawn (feat. Sia) - Radio Edit - https://open.spotify.com/track/1j4kHkkpqZRBwE0A4CN4Yv
Happy Now - https://open.spotify.com/track/4keoy2fqgwGnbWlm3ZVZFa
Torn - Bonus Track - https://open.spotify.com/track/7lwYNEUrDP4thCD6nQ4nBv
NDA - https://open.spotify.com/track/38GBNKZUhfBkk3oNlWzRYd
Starboy - https://open.spotify.com/track/5aAx2yezTd8zXrkmtK

# And for those played primarily in the USA

In [72]:
pl.Config.set_tbl_rows(10)
usa = (df
          #  .pipe(wcs_specific)
          .filter(pl.col('regions') == 'USA')
          .select('song', 'num_djs', 'num_playlists', 'num_regions', 'regions')
          .unique()
          .sort('num_djs', descending=True)
          )

display(usa)

for song, *_ in usa.head(500).iter_rows():
  print(song)

song,num_djs,num_playlists,num_regions,regions
str,u32,u32,u32,str
"""Nightcall - https://open.spotify.com/track/5lCJIgiqGmkc7uP6YJ0COc""",14,17,2,"""USA"""
"""Lady Luck - https://open.spotify.com/track/1AORfnD538aKwUzSyqTz05""",12,15,2,"""USA"""
"""Vibe - https://open.spotify.com/track/6M2aVK2M2cj7ywBis4tkMH""",12,26,2,"""USA"""
"""Promiscuous - https://open.spotify.com/track/3SwlakM6VX47IwG0Wll5ek""",12,17,2,"""USA"""
"""Sexy And I Know It - https://open.spotify.com/track/7C2kIGrflgP2RKqMuf9GJR""",12,25,2,"""USA"""
…,…,…,…,…
"""Bring the Noise Remix - Benny Benassi Pump-kin remix - https://open.spotify.com/track/5XziNvQ99xc0PW…",1,2,1,"""USA"""
"""All At Sea - https://open.spotify.com/track/6o3LfBHc7yhr5emXRFJDWG""",1,1,1,"""USA"""
"""I Want You Around - https://open.spotify.com/track/2tUuvamTbprsZ2YevAuzGC""",1,3,1,"""USA"""


Nightcall - https://open.spotify.com/track/5lCJIgiqGmkc7uP6YJ0COc
Lady Luck - https://open.spotify.com/track/1AORfnD538aKwUzSyqTz05
Vibe - https://open.spotify.com/track/6M2aVK2M2cj7ywBis4tkMH
Promiscuous - https://open.spotify.com/track/3SwlakM6VX47IwG0Wll5ek
Sexy And I Know It - https://open.spotify.com/track/7C2kIGrflgP2RKqMuf9GJR
Blackout - https://open.spotify.com/track/3Yr4gXagnFlMVGpMWAkOPa
I Took A Pill In Ibiza - Seeb Remix - https://open.spotify.com/track/1MtUq6Wp1eQ8PC6BbPCj8P
Heartbreak Warfare - https://open.spotify.com/track/4gs07VlJST4bdxGbBsXVue
Miss You - https://open.spotify.com/track/6giHzoNmL0kwyBZAXGIIc2
Forget You - https://open.spotify.com/track/7AqISujIaWcY3h5zrOqt5v
Ring - https://open.spotify.com/track/11Ey4yJVeWEsA73KjACkVY
Cheap Sunglasses - https://open.spotify.com/track/4MatjVmXUxuBf7WeInnRBG
Hold Up, Wait a Minute - https://open.spotify.com/track/1iuSdhY8eRs3Nx1phaQy6J
Big Girl Now - https://open.spotify.com/track/2Tkq5t9RZQubezdV9jN3cd
Space Ghost (Coast

# And for those played primarily in Asia

In [73]:
pl.Config.set_tbl_rows(10)
asia = (df
          #  .pipe(wcs_specific)
          .filter(pl.col('regions') == 'Asia')
          .select('song', 'num_djs', 'num_playlists', 'num_regions', 'regions')
          .unique()
          .sort('num_djs', descending=True)
          )

display(asia)

for song, *_ in asia.head(500).iter_rows():
  print(song)

song,num_djs,num_playlists,num_regions,regions
str,u32,u32,u32,str
"""Can We Kiss Forever? - https://open.spotify.com/track/58wyJLv6yH1La9NIZPl3ne""",4,4,2,"""Asia"""
"""Give Me One Reason - Live - https://open.spotify.com/track/6jTqomGlRFQ52EgIIWBfnY""",4,5,2,"""Asia"""
"""Never Gonna Give You Up - https://open.spotify.com/track/4cOdK2wGLETKBW3PvgPWqT""",4,4,2,"""Asia"""
"""Bo-Time - https://open.spotify.com/track/6QWeuQ8FUcaySjBzOpQJqe""",4,6,2,"""Asia"""
"""Little Bit More - https://open.spotify.com/track/6rG1DdPCTzGibgHliYJ4ws""",3,4,2,"""Asia"""
…,…,…,…,…
"""Fantasy - TEEMID Remix - https://open.spotify.com/track/2iepE5uJeuBJnkg6ckQBqN""",1,1,1,"""Asia"""
"""Blame It (feat. T-Pain) - Radio Mix - https://open.spotify.com/track/6rscwwM1kf6XjsRbB2vgJ4""",1,3,1,"""Asia"""
"""All I Ever Need - https://open.spotify.com/track/0YEsfwQpG7ofdIiHWA0dHi""",1,1,1,"""Asia"""


Can We Kiss Forever? - https://open.spotify.com/track/58wyJLv6yH1La9NIZPl3ne
Give Me One Reason - Live - https://open.spotify.com/track/6jTqomGlRFQ52EgIIWBfnY
Never Gonna Give You Up - https://open.spotify.com/track/4cOdK2wGLETKBW3PvgPWqT
Bo-Time - https://open.spotify.com/track/6QWeuQ8FUcaySjBzOpQJqe
Little Bit More - https://open.spotify.com/track/6rG1DdPCTzGibgHliYJ4ws
Female - https://open.spotify.com/track/6p01Q7WF857BsAsByBXyaG
Irreplaceable - https://open.spotify.com/track/07lxDm1s8FVO4GF54Nooiz
I'll Be Home - https://open.spotify.com/track/0lTurWECr2XL607eJAj1A3
Rewrite The Stars - https://open.spotify.com/track/7FlHNJT4TC120CDvFOHzei
Hey Daddy (Daddy's Home) - https://open.spotify.com/track/030OCtLMrljNhp8OWHBWW3
I'm with You - https://open.spotify.com/track/1jlG3KJ3gdYmhfuySFfpO1
Needed - https://open.spotify.com/track/5MHHcFRlO9LUP153GFHOk6
I'll Be Around (feat. Timbaland) - Club Mix - https://open.spotify.com/track/1K8ppGlQShR07EuUcR4eGO
Somebody Else - https://open.spotify

# And for those played primarily in MENA

In [74]:
pl.Config.set_tbl_rows(10)
mena = (df
          #  .pipe(wcs_specific)
          .filter(pl.col('regions') == 'MENA')
          .select('song', 'num_djs', 'num_playlists', 'num_regions', 'regions')
          .unique()
          .sort('num_djs', descending=True)
          )

display(mena)

for song, *_ in mena.head(500).iter_rows():
  print(song)

song,num_djs,num_playlists,num_regions,regions
str,u32,u32,u32,str
"""Mary Had a Little Lamb - https://open.spotify.com/track/50yVEqOnLHtRTHWq3PWBuZ""",6,6,2,"""MENA"""
"""Where Have You Been - https://open.spotify.com/track/1tjY4H0X706ZH03SXa1DFh""",6,6,2,"""MENA"""
"""Titanium - feat. Sia - https://open.spotify.com/track/1x6ACsKV4UdWS2FMuPFUiT""",6,6,2,"""MENA"""
"""Turn Me On - feat. Nicki Minaj - https://open.spotify.com/track/39w8X3cyEiM4uxObz55Qm5""",6,6,2,"""MENA"""
"""Can't Say No - https://open.spotify.com/track/0AHpuVdVEbQu8bzGUieSKi""",6,6,2,"""MENA"""
…,…,…,…,…
"""Intro (Call to Hearts) - https://open.spotify.com/track/33MD31lMoZg7TTzgwsjtGB""",1,1,1,"""MENA"""
"""SaxoSuelta Oriente Tribe - https://open.spotify.com/track/34inPy9EJX1LijFMKdpTs3""",1,1,1,"""MENA"""
"""Back in my Life - https://open.spotify.com/track/7HhZ6BEpz5RFYXJn3TCLMg""",1,1,1,"""MENA"""


Mary Had a Little Lamb - https://open.spotify.com/track/50yVEqOnLHtRTHWq3PWBuZ
Where Have You Been - https://open.spotify.com/track/1tjY4H0X706ZH03SXa1DFh
Titanium - feat. Sia - https://open.spotify.com/track/1x6ACsKV4UdWS2FMuPFUiT
Turn Me On - feat. Nicki Minaj - https://open.spotify.com/track/39w8X3cyEiM4uxObz55Qm5
Can't Say No - https://open.spotify.com/track/0AHpuVdVEbQu8bzGUieSKi
La Bachata - https://open.spotify.com/track/5ww2BF9slyYgNOk37BlC4u
Cry Baby - https://open.spotify.com/track/7oDSG2Q9RBU0QjgtfYQOdD
Monster (Shawn Mendes & Justin Bieber) - https://open.spotify.com/track/1HbA4N1MiOsPthALesGFR1
Wake Me Up Before You Go-Go - https://open.spotify.com/track/5XsMz0YfEaHZE0MTb1aujs
Here I Go Again - https://open.spotify.com/track/1tQ5TSr1tyeQUbHKBee0jv
Sin Salsa No Hay Paraíso - https://open.spotify.com/track/714JJ27SmuQEghyVauzMPj
Wild Wild Love (feat. G.R.L.) - https://open.spotify.com/track/6DcDdDevI94Dh4vc5anXBE
Take On Me - https://open.spotify.com/track/1nbMkRhfldXcmCyB83